In [1]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_md")

def tokenizer(id,fText,l1,l2):
    doc = nlp(fText)
    bdoc = nlp(l1)
    mdoc = nlp(l2)
    ### TOKENIZING ###, ### REMOVING STOP WORDS ### if not token.is_stop
    wList = [token for token in doc]
    bList = [token for token in bdoc]
    mList = [token for token in mdoc]
    ### NORMALIZING ###, ### STEMMING ###, ### LEMMATIZATION ###
    # wList = [token.lemma_ for token in wList]
    ### POS filtering ###
    df = pd.DataFrame(columns=['satzId','Wort','Attribut'], data = {'satzId': id, 'Wort': wList})
    #cast falls nötig von spacy token zu string konvertieren
    df['Wort'] = df['Wort'].astype(str)
    df['Attribut'] = "O"

    #search for all times the first word of the brand occures in the text
    b_brand = df.loc[df['Wort'].str.lower() == str(bList[0]).lower(), 'Attribut']

    #print(b_brand.index.asi8)

    # searching for the Brand Labels

    #if(len(b_brand.index.asi8) == 1):
    #    df.at[b_brand.index.asi8[0], 'Attribut'] = "B-Brand"
    if(len(bList)==1):
        df.loc[df['Wort'].str.lower() == str(bList[0]).lower(), 'Attribut'] = 'B-Brand'
    else:
        for x in range(len(b_brand.index.asi8)):
            #everytime the first Word of the Brand occurs in the Text we need to ckeck the following tokens if it really is the whole Brand Label
            df_to_check = df.iloc[b_brand.index.asi8[x]:b_brand.index.asi8[x]+len(bList)]
            #print(df_to_check)
            # amount of hits have to be the length of the Brand
            hits = 0
            for i, row in enumerate(df_to_check['Wort']):
                if(row.lower() == str(bList[i]).lower()): #hit if the word in the token is the same as in the brand label we search
                    hits += 1
                else:
                    break
            if(hits == len(bList)):
                df.iloc[b_brand.index.asi8[x]:b_brand.index.asi8[x]+len(bList)]['Attribut'] = "I-Brand"
                df.at[b_brand.index.asi8[x], 'Attribut'] = "B-Brand"
                df.at[b_brand.index.asi8[x]+len(bList)-1, 'Attribut'] = "E-Brand"

    #search for all times the first word of the brand occures in the text
    b_modelnumber = df.loc[df['Wort'].str.lower() == str(mList[0]).lower(), 'Attribut']


    if(len(mList) == 1):
        df.loc[df['Wort'].str.lower() == str(mList[0]).lower(), 'Attribut'] = "B-Modelnumber"
    else:
        for x in range(len(b_modelnumber.index.asi8)):
            #everytime the first Word of the Brand occurs in the Text we need to ckeck the following tokens if it really is the whole Brand Label
            df_to_check = df.iloc[b_modelnumber.index.asi8[x]:b_modelnumber.index.asi8[x]+len(mList)]
            #print(df_to_check)
            # amount of hits have to be the length of the Brand
            hits = 0
            for i, row in enumerate(df_to_check['Wort']):
                if(row.lower() == str(mList[i]).lower()): #hit if the word in the token is the same as in the brand label we search
                    hits += 1
                else:
                    break
            if(hits == len(mList)):
                df.iloc[b_modelnumber.index.asi8[x]:b_modelnumber.index.asi8[x]+len(mList)]['Attribut'] = "I-Modelnumber"
                df.at[b_modelnumber.index.asi8[x], 'Attribut'] = "B-Modelnumber"
                df.at[b_modelnumber.index.asi8[x]+len(mList)-1, 'Attribut'] = "E-Modelnumber"


    #for each in bList:
    #    df.loc[df['Wort'] == str(each), 'Attribut'] = "B-Brand"
    return df

In [2]:
ecsv = pd.read_csv('Daten/KoepckeEigen/electronicFixed.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
ecsv.dropna(axis = 0)
dList = []
df = pd.DataFrame()
for index, row in ecsv.iterrows():
    df = df.append(tokenizer(index, str(row['name']) + str(row['productdescription']), str(row['brand']), str(row['modelnumber'])))
    #df = df.append(tokenizer(index, str(row['name']), str(row['brand']), str(row['modelnumber'])))
    if index % 1000 == 0:
        print('Verarbeitete Produkte:', index)
        df.to_csv('Daten/trainingData.csv', index = False)
#print(dList)
#print(dList.pop(0))
#df = pd.DataFrame(dList, columns=['Id', 'satzID', 'Wort', 'Attribut'])
print(df)
df.to_csv('Daten/trainingData.csv', index = False)

Verarbeitete Produkte:  0
<ipython-input-1-b6fa3d8a6065>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[b_brand.index.asi8[x]:b_brand.index.asi8[x]+len(bList)]['Attribut'] = "I-Brand"
<ipython-input-1-b6fa3d8a6065>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[b_modelnumber.index.asi8[x]:b_modelnumber.index.asi8[x]+len(mList)]['Attribut'] = "I-Modelnumber"
Verarbeitete Produkte:  1000
Verarbeitete Produkte:  2000
Verarbeitete Produkte:  3000
Verarbeitete Produkte:  4000
Verarbeitete P